### Поиск похожих тегов

Задачей этого проетка является поиск похожих тегов на сайте "Хабрахабр".
Данными являются 134 тысячи постов, выкаченных с этого сайта.
Стоит отметить, что на Хабрахабре теги расстваляются свободно, то есть нет заранее определенного их множества, и пользователь может создать новый. Из-за этого общее число тегов на сайте сравнимо с количеством статей, поэтому эксперимент был ограничен пятью тысячами самых популярных тегов.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [2]:
import json

df = pd.DataFrame(list(map(json.loads, tqdm(open("howpop_train.jsonlines")))))

134137it [01:32, 1456.99it/s]


In [35]:
df['tags'].tail()

134132              [json, стандарты, никто не читает теги]
134133               [астероиды, земля, астероидная угроза]
134134                 [арзамас, ркн, медуза, эраст перцов]
134135    [регулирование интернета, очередной законопрое...
134136                        [React.js, Javascript, forms]
Name: tags, dtype: object

### TfIdf и похожесть по косинусу

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# превращаем лист тегов в строку, где теги разделены запятой
tags_strings = []
for i in range(len(df)):
    tags_strings.append(", ".join(df["tags"][i]))

# шаблон, по которому TfidfVectorizer будет брать токены
token_pattern = r'\b([^,]+)'

vectorizer = TfidfVectorizer(token_pattern=token_pattern, max_features=5000)

# matrix - матрица: строки - объекты, столбцы - теги
matrix = vectorizer.fit_transform(tags_strings)

# matrix_T - матрица: строки - теги, столбцы - объекты
matrix_T = matrix.toarray().transpose()

# определяем "похожесть" тегов друг на друга
s = cosine_similarity(matrix_T)

In [4]:
# {индекс тега в таблице коэффициентов "похожестей": название тега}
tags_idx = {vectorizer.vocabulary_[tag]: tag for tag in vectorizer.vocabulary_.keys()}

In [54]:
# {тег: лист тегов, похожих на него}
similar_tags = {}

In [55]:
for i in tags_idx.keys():
    most_similar = list(np.where(s[i] > 0.09)[0])
    tags = []
    for idx in most_similar:
        tags.append(tags_idx[idx])
    similar_tags[tags_idx[i]] = tags

In [56]:
i = 0
for key in similar_tags.keys():
    print(key, ':', similar_tags[key])
    i += 1
    if i == 25: break

webasyst : ['webasyst']
wpf : ['binding', 'c#', 'mvvm', 'net', 'silverlight', 'winforms', 'wpf', 'xaml']
san : ['iscsi', 'san', 'системы хранения данных', 'схд']
gns3 : ['cisco', 'gns3', 'сети для самых маленьких']
патент : ['изобретение', 'патент', 'патентование']
css3 : ['css', 'css 3', 'css3', 'css3 animation', 'css3 transition', 'html', 'html5', 'js', 'веб-разработка', 'дайджест', 'интересное', 'ресурсы', 'ссылки', 'ссылки на сми']
ford : ['ford']
игровые консоли : ['игровые консоли', 'игровые приставки', 'приставки', 'эмуляторы']
киев : ['киев', 'хабравстречи']
микроконтроллер : ['микроконтроллер']
виртуализация : ['hyper-v', 'vdi', 'vmware', 'xen', 'виртуализация', 'виртуальная машина', 'гипервизор']
импортозамещение по : ['законодательство и ит', 'импортозамещение по', 'минкомсвязи']
nagios : ['nagios', 'мониторинг']
cloud : ['cloud', 'cloud computing', 'microsoft azure', 'облака', 'облако']
опыты : ['goldphone', 'gtv', 'simple science', 'для детей', 'наука', 'научно-популярное'

### Word2Vec

In [7]:
# лист всех постов Хабрахабра
corpus = list(df.content.values)

In [29]:
# лист из пяти тысяч самых популярных тегов, очищенных от чисел
tags = list(similar_tags.keys())
for i in range(len(tags)):
    tags[i] = re.sub('\d+', '', tags[i])
tags = list(filter(None, list(set(tags))))

Обработка поста:
1. Замена всех пробельных символов на пробел.
2. Удаление всех html-тегов.
3. Удаление знаков пунктуации.
4. Если какой-то из тегов встречается в статье, то в нем заменются все пробелы знаком подчеркивания.
5. Разделение поста на лист по знакам препинания.

In [36]:
import re
from bs4 import BeautifulSoup

def save_tags(text):
    text = re.sub("\s", " ", text).lower()
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text = re.sub('[\'|"|)|(|}|{|+|*|~|\d|“|…]*', '', text)
    for tag in tags:
        if tag.lower() in text:
            text = text.replace(tag.lower(), tag.lower().replace(' ', '_'))
    return list(filter(None, re.split('[ |.|,|;|:|/|!|?|—]', text)))

In [38]:
import joblib
sentences = joblib.Parallel(n_jobs=2)(tqdm(list(map(joblib.delayed(save_tags),corpus))))

 51%|█████▏    | 68962/134137 [12:43<14:54, 72.86it/s]/Users/Julia/anaconda/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'/////////'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
100%|██████████| 134137/134137 [31:21<00:00, 41.30it/s]


Обучение модели Word2Vec.

In [40]:
import gensim
from gensim.models import Word2Vec

num_features = 128    # Word vector dimensionality
num_workers = 2      # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

model = Word2Vec(sentences, workers=num_workers, size=num_features, \
                 window = context, sample = downsampling, seed=1, null_word="NULL")

In [8]:
# лист тегов
columns = list(similar_tags.keys())

In [40]:
import re

#{тег: [[другой тег, мера похожести этих тегов], ...]}
w2v_tag_closeness = {}

for tag in tqdm(columns):
    w2v_tag_closeness[tag] = []
    first = re.sub('\d+', '', tag).lower().replace(' ', '_')
    
    for another in columns:
        second = re.sub('\d+', '', another).lower().replace(' ', '_')
        try:
            w2v_tag_closeness[tag].append([another, model.similarity(first, second)])
        except:
            pass

100%|██████████| 5000/5000 [07:49<00:00, 12.25it/s]


In [49]:
i = 0
for key in w2v_tag_closeness:
    w2v_tag_closeness[key].sort(key=lambda x: -x[1])
    try:
        print(key, ':', [w2v_tag_closeness[key][i][0] for i in range(4)])
        i += 1
    except:
        pass
    if i == 20: break

webasyst : ['webasyst', '1с-битрикс', 'phpshop', 'wix']
wpf : ['wpf', 'winforms', 'silverlight', 'winrt']
san : ['san', 'iscsi', 'netapp fas', 'netapp']
gns3 : ['gns3', 'qemu', 'virtualbox', 'kvm']
css3 : ['css3', 'css', 'css 3', 'sass']
ford : ['ford', 'honda', 'nissan', 'toyota']
киев : ['киев', 'минск', 'харьков', 'новосибирск']
микроконтроллер : ['микроконтроллер', 'atmega8', 'atmega', 'attiny13']
high frequency trading : ['high frequency trading', 'hft', 'высокочастотный трейдинг', 'трейдинг']
nagios : ['nagios', 'zabbix', 'puppet', 'systemd']
cloud : ['cloud', '1cloud', 'microsoft azure', 'azure']
опыты : ['опыты', 'эксперименты', 'испытания', 'научные исследования']
china : ['china', 'verizon', 'at&t', 'вымпелком']
frontend : ['frontend', 'backend', 'front-end', 'фронтенд']
driver : ['driver', 'firmware', 'remote control', 'hardware']
резервное копирование файлов : ['резервное копирование файлов', 'продвижение в интернете', 'мониторинг сервера', 'проект око']
ddd : ['ddd', 'uml'

Небольшие эксперименты на сравнение

In [69]:
w2v_tag_closeness['css'].sort(key=lambda x: -x[1])
print('TfIdf | Word2Vec')
print('----------------')
for i in range(10):
    print(similar_tags['css'][i], '|', w2v_tag_closeness['css'][i][0])

TfIdf | Word2Vec
----------------
css | css3
css3 | css
html | css 3
js | sass
less | html
верстка | html5
вёрстка | scss
интересности&полезности | svg
ресурсы | javascript
ссылки | flexbox


In [70]:
w2v_tag_closeness['stanford'].sort(key=lambda x: -x[1])
print('TfIdf | Word2Vec')
print('----------------')
for i in range(2):
    print(similar_tags['stanford'][i], '|', w2v_tag_closeness['stanford'][i][0])

TfIdf | Word2Vec
----------------
stanford | stanford
udacity | coursera


In [74]:
w2v_tag_closeness['python'].sort(key=lambda x: -x[1])
print('TfIdf | Word2Vec')
print('----------------')
for i in range(len(similar_tags['python'])):
    print(similar_tags['python'][i], '|', w2v_tag_closeness['python'][i][0])

TfIdf | Word2Vec
----------------
django | python
flask | python3
python | perl6
python3 | perl


In [75]:
w2v_tag_closeness['deep learning'].sort(key=lambda x: -x[1])
print('TfIdf | Word2Vec')
print('----------------')
for i in range(len(similar_tags['deep learning'])):
    print(similar_tags['deep learning'][i], '|', w2v_tag_closeness['deep learning'][i][0])

TfIdf | Word2Vec
----------------
deep learning | deep learning
глубокое обучение | machine learning


Оба метода работают и дают хороший результат. Какой метод дает лучший результат сказать сложно.
Модель Word2Vec легка "в управлении", то есть с её помощью не надо думать об индексах в матрицах. С другой стороны, TfIdf обучается значительно быстрее и не требует больших мощностей.
Основная сложность проекта заключалась в вопросах: как наилучшим образом обработать статьи для подачи в модель Word2Vec, и как создать эмбеддинги тегов с её помощью.
Для дальнейших исследований можно использовать оба варианта.